#About
In this notebook, we process the ```.psl```-files of blat. We have allready processed these files, keeping only the best match for each contigs.

In [1]:
import os, re

Make list of substrain that are within one strain. These PSL-files will be ignored.

In [2]:
f = open('/Users/robin/MEGA/Work/Hubrecht/Unmapped/strainSpecAnalysis/500plus/commonMouseDNA.psl','r')
seenContigs = set([])  # all contigs allready stored in a key and/or value
dikt = {} # key = q-name, val = set of t-names
linecounter = 0
for line in f:
    linecounter += 1
    if (linecounter % 100000) == 0:
        print linecounter
    fields = re.split(r'\t+', line.rstrip())
    match = float(fields[0])
    qsize = float(fields[10])
    qname = str(fields[9])
    tname = str(fields[13])
    if ( (match/(qsize-float(fields[3]))) > .9 ):
        if (qname != tname):
            if qname not in seenContigs:
                if qname not in dikt.keys() and tname not in dikt.keys():
                    dikt[qname] = set([])
                    dikt[qname].add(tname)
                elif qname in dikt.keys() and tname not in dikt.keys():
                    dikt[qname].add(tname)
                seenContigs.add(tname)
f.close()

100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000


In [ ]:
os.system("rm tmp*")
os.system("rm consensus.fa")
clusterCount = 0
for k,v in dikt.items():
    clusterCount += 1
    rat = float(clusterCount)/len(dikt)
    if (rat % 0.01) == 0:
        print str(rat * 100) + "%"
    outfile = open('tmp.lst', 'w')
    outfile.write("\n".join(v))
    outfile.write("\n" + k + "\n")
    outfile.close()
    os.system("~/bin/seqtk-master/seqtk subseq /Users/robin/MEGA/Work/Hubrecht/Unmapped/strainSpecAnalysis/500plus/commonMouseDNA.fa tmp.lst | sed 's/^>/>cluster_" + str(clusterCount) + "_/' > tmp.fa")
    os.system("~/bin/clustalo -i tmp.fa --full-iter -o tmp_clustalo --outfmt=clu --force")
    os.system("cp tmp_clustalo tmp.clustalo")
    os.system("perl ~/bin/ANDES/ClustalALN_to_PositionProfile.pl -a tmp.clustalo")
    os.system("perl ~/bin/ANDES/Profile_To_ConsensusFASTA.pl -c tmp.consensus -p tmp.clustalo")
    os.system("sed 's/^>/>cluster_" + str(clusterCount) + "/' tmp.consensus >> consensus.fa")
f.close()